In [ ]:
%env LD_LIBRARY_PATH /usr/local/cuda-10.0/lib64/

In [ ]:
import os
from keras import backend as K
import tensorflow as tf
import keras as ks
from keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
from keras import layers
from keras import models
from keras.preprocessing import image
import numpy as np
from keras import optimizers
import matplotlib.pyplot as plt
from time import time
from notebook_utils import *

tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

config = tf.ConfigProto(log_device_placement=True)
config.gpu_options.allow_growth=True
sess = tf.Session(config=config)
K.set_session(sess)

if tf.test.gpu_device_name():
 print("Default GPU Device: {}".format(tf.test.gpu_device_name()))
else:
 print("Please install GPU version of TF")

In [ ]:
base_dir = '/home/benoitpatra/code/twins-recognizer/data/sets_faked'

TRAIN_DIR = os.path.join(base_dir, 'train')
VAL_DIR = os.path.join(base_dir, 'validation')
TEST_DIR = os.path.join(base_dir, 'test')

TARGET_SIZE = 200
BATCH_SIZE = 10
TRAIN_SET_SIZE=600
steps_per_epoch= TRAIN_SET_SIZE // BATCH_SIZE

In [ ]:
## Define TRAINING SET
train_datagen =  ImageDataGenerator(rescale=1./255, rotation_range=0,
                              width_shift_range=0.2,
                              height_shift_range=0.2,
                              shear_range=0.2,
                              zoom_range=0.2,
                              horizontal_flip=True, fill_mode='nearest')


train_generator = train_datagen.flow_from_directory(TRAIN_DIR, 
                                                    target_size=(TARGET_SIZE, TARGET_SIZE), 
                                                    batch_size = BATCH_SIZE, 
                                                    class_mode='binary')

In [ ]:
## Define VALIDATION SET
validation_datagen = ImageDataGenerator(rescale=1./255)
validation_generator = validation_datagen.flow_from_directory(VAL_DIR, 
                                                    target_size=(TARGET_SIZE, TARGET_SIZE), 
                                                    batch_size = BATCH_SIZE, 
                                                    class_mode='binary')

In [ ]:
test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow_from_directory(TEST_DIR, 
                                                    target_size=(TARGET_SIZE, TARGET_SIZE), 
                                                    batch_size = BATCH_SIZE, 
                                                    class_mode='binary')

In [ ]:
img_index=7
img = next(train_generator)
print(img[0][img_index,:,:,:].shape)
print_image_from_gen(img[0][img_index,:,:,:])
print(img[1][img_index])

In [ ]:
def train_and_validate_model(model):
    t1 = time()
    model.compile(loss='binary_crossentropy', optimizer=optimizers.RMSprop(lr=1e-3), metrics=['acc'])
    history = model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=30, validation_data=validation_generator, validation_steps=50)
    t2 = time()
    elapsed = t2 - t1
    print('Elapsed time is %f seconds.' % elapsed)
    plot_history2(model.history)

def model_dense():
    model = models.Sequential()
    model.add(layers.Dense(128, activation='relu', input_shape=(TARGET_SIZE,TARGET_SIZE, 3)))
    model.add(layers.Dropout(0.5))
    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.Dropout(0.5))
    model.add(layers.Flatten())
    model.add(layers.Dense(1, activation='sigmoid'))
    return model

In [ ]:
from adhoc_convnet import *
from VGG16_base import *

#print("---NAIVE DENSE---")
#naive_dense = model_dense()
#naive_dense.summary()

print("---CONVENET---")
model_convenet_ad_hoc = adhoc_convnet(TARGET_SIZE)
model_convenet_ad_hoc.summary()

print("---VGG16 BASE---")
vgg_model = vgg16_base(TARGET_SIZE)
vgg_model.summary()


print("---EVALUATE NAIVE DENSE--")
train_and_validate_model(naive_dense)

In [ ]:
print("---EVALUATE CONVNET ADHOC--")
train_and_validate_model(model_convenet_ad_hoc)

In [ ]:
print("---EVALUATE VGG BASE--")
train_and_validate_model(vgg_model)

In [ ]:
batch = next(test_generator)
view_prediction(vgg_model, batch)

In [ ]:
model_convenet_ad_hoc.save_weights("model_convnet_20200421.h5")